# Comparaison entre le prix du marché d'une option et l'évaluation d'une option avec Monte Carlo et Black-Scholes

## Objectif général
L'objectif de cet exercice est de déterminer si le prix d'une option européenne sélectionnée est surévalué ou sous-évalué en utilisant deux méthodes : la simulation de Monte Carlo et la formule de Black-Scholes.

## Structure du Notebook :

### Fonctions

1. **Téléchargement et validation des données :**
   - Validation du ticker
   - Téléchargement des prix historiques et gestion des valeurs manquantes

2. **Estimation des paramètres statistiques et simulation de Monte Carlo**
   - Calcul des rendements logarithmiques et des paramètres statistiques
   - Simulation de Monte Carlo

3. **Formule de Black-Scholes**
   - Calcul du prix théorique d'une option avec la formule de Black-Scholes

### Main

# Importation des bibliothèques nécessaires

In [46]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import tabulate as tb
import math
import datetime as dt

# Fonctions

## 1. Téléchargement et validation des données 

### A. Validation du ticker

Fonction permettant de vérifier la validité d'un ticker saisi par l'utilisateur (par exemple, AAPL ou MSFT) à l'aide de yfinance. Le ticker est retourné uniquement si des données valides sont disponibles, sinon l'utilisateur est invité à saisir un autre ticker.

In [48]:
def validate_ticker(default="AAPL"):
    while True:
        ticker = input(f"Entrez le ticker (par ex. AAPL, MSFT, TSLA) (par défaut : {default}) : ").strip().upper()
        ticker = ticker if ticker else default
        stock = yf.Ticker(ticker)
        stock.history(period="1d")
        prices = stock.history(period="1d")
        if not prices.empty:
            return ticker  
        print(f"Le ticker {ticker} n'est pas valide. Veuillez réessayer.")

### B. Téléchargement des prix historiques et gestion des valeurs manquantes

Fonction permettant de télécharger les données de prix de clôture pour une action donnée sur une période spécifiée en utilisant yfinance. Si des données sont absentes ou invalides, un message d'erreur est affiché, et une exception est levée. En cas de valeurs manquantes dans les données téléchargées, celles-ci sont remplacées à l'aide des méthodes ffill() et bfill() pour assurer la continuité. Une fois les données valides, elles sont retournées sous forme de DataFrame.

In [20]:
def download_prices(ticker, start_date, end_date):
    print(f"Téléchargement des données pour {ticker} de {start_date} à {end_date}...")
    prices = yf.download(ticker, start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'))[['Close']]
    if prices.empty:
        raise ValueError(f"Aucune donnée téléchargée pour {ticker}. Vérifiez les dates ou le ticker.")
    if prices.isnull().values.any():
        print("Attention : Des valeurs manquantes ont été détectées. Elles seront remplacées par ffill() et bfill().")
        prices = prices.ffill().bfill()
    print(f"Données téléchargées avec succès pour {ticker}.")
    return prices

## 2. Simulation de Monte Carlo 

### A. Calcul des rendements logaritmiques et de paramètres statistiques

Fonction permettant de calculer les rendements logarithmiques d'une action sur une période donnée, ainsi que leur moyenne et leur écart-type. Si une sous-période est spécifiée, les données de prix sont filtrées en conséquence. En cas d'absence de données pour la période sélectionnée, une exception est levée. Les rendements sont calculés en prenant la différence logarithmique des prix de clôture consécutifs, après suppression des valeurs manquantes. La fonction retourne les rendements sous forme de série, leur moyenne et leur écart-type.

In [21]:
def calculate_stat_returns(prices, start_sub_period, end_sub_period):
    sub_prices = prices.loc[start_sub_period:end_sub_period]
    if sub_prices.empty:
        raise ValueError(f"No data available for the period {start_sub_period} to {end_sub_period}.")
    returns = np.log(sub_prices['Close']).diff().dropna()
    return returns, returns.mean(), returns.std()

### B. Simulation de Monte Carlo

Fonction permettant de simuler des trajectoires de prix futurs pour un actif donné à l'aide de la méthode Monte Carlo. À partir du prix initial \( S \), du strike \( K \), de la volatilité \( sigma \), du taux sans risque \( r \), et du temps jusqu'à maturité \( T \), un nombre spécifié de simulations \( nb\_simulation \) est généré. Chaque simulation correspond à une trajectoire calculée sur une base journalière à l'aide d'une formule log-normale.
Les payoffs (gains potentiels à l'échéance) sont calculés pour chaque trajectoire en fonction du type d'option choisi ("call" ou "put"). Le prix moyen actualisé des payoffs est ensuite retourné, correspondant à l'estimation Monte Carlo du prix de l'option.

In [22]:
def monte_carlo_simulation(S, K, T, r, sigma, nb_simulation=1000, option_type="call"):
    dt = T / 252
    payoffs = []
    for _ in range(nb_simulation):
        price = S
        for _ in range(int(T * 252)):
            price *= np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * np.random.normal())
        if option_type.lower() == "call":
            payoff = max(price - K, 0)
        elif option_type.lower() == "put":
            payoff = max(K - price, 0)
        payoffs.append(payoff)
    return np.mean(payoffs) * np.exp(-r * T)

## 3. Formule de Black-Scholes

### A. Calcul du prix théorique d'une option avec la formule de Black-Scholes

Fonction calculant le prix d'une option européenne (Call ou Put) à l'aide de la formule de Black-Scholes. Elle utilise le prix actuel de l'actif \(S\), le prix d'exercice \(K\), le temps à maturité \(T\), le taux sans risque \(r\) et la volatilité \(sigma\). Le calcul repose sur les probabilités cumulées normales \(Nd_1\) et \(Nd_2\), retournant le prix actualisé de l'option selon le type spécifié.

In [23]:
def black_scholes(S, K, T, r, sigma, option_type):
    if T <= 0:
        raise ValueError("Time to maturity (T) must be positive.")
    
    d1 = (math.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)

    if option_type.lower() == "call":
        return S * norm.cdf(d1) - K * math.exp(-r * T) * norm.cdf(d2)
    elif option_type.lower() == "put":
        return K * math.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    else:
        raise ValueError("option_type must be 'call' or 'put'.")

# Main

In [24]:
# Étape 1 : Choix et validation du ticker
ticker = validate_ticker()
stock = yf.Ticker(ticker)


In [25]:
# Étape 2 : Téléchargement des données historiques
today = dt.datetime.today()
default_start_date = today - dt.timedelta(days=365)  # Default: past year
prices = download_prices(ticker, default_start_date, today)

[*********************100%***********************]  1 of 1 completed

Téléchargement des données pour AAPL de 2023-12-10 22:37:14.672486 à 2024-12-09 22:37:14.672486...
Données téléchargées avec succès pour AAPL.


In [26]:
# Étape 3 : Sélection d'une date d'expiration
expiration_dates = stock.options
while True:
    try:
        data_list = [[i + 1, date] for i, date in enumerate(expiration_dates)]
        headers = ["No.", "Expiration Date"]
        print(f"\nAvailable expiration dates for {ticker}:")
        print(tb.tabulate(data_list, headers=headers, tablefmt="fancy_grid"))
        selected_date = expiration_dates[int(input("Choose an expiration date (by No.) (default: 1): ") or 1) - 1]
        if pd.to_datetime(selected_date) <= today:
            raise ValueError("Selected expiration date must be in the future.")
        print(f"Selected expiration date: {selected_date}")
        break
    except (IndexError, ValueError) as e:
        print(f"Error: {e}. Please enter a valid number.")


Available expiration dates for AAPL:
╒═══════╤═══════════════════╕
│   No. │ Expiration Date   │
╞═══════╪═══════════════════╡
│     1 │ 2024-12-13        │
├───────┼───────────────────┤
│     2 │ 2024-12-20        │
├───────┼───────────────────┤
│     3 │ 2024-12-27        │
├───────┼───────────────────┤
│     4 │ 2025-01-03        │
├───────┼───────────────────┤
│     5 │ 2025-01-10        │
├───────┼───────────────────┤
│     6 │ 2025-01-17        │
├───────┼───────────────────┤
│     7 │ 2025-01-24        │
├───────┼───────────────────┤
│     8 │ 2025-02-21        │
├───────┼───────────────────┤
│     9 │ 2025-03-21        │
├───────┼───────────────────┤
│    10 │ 2025-04-17        │
├───────┼───────────────────┤
│    11 │ 2025-06-20        │
├───────┼───────────────────┤
│    12 │ 2025-07-18        │
├───────┼───────────────────┤
│    13 │ 2025-08-15        │
├───────┼───────────────────┤
│    14 │ 2025-09-19        │
├───────┼───────────────────┤
│    15 │ 2025-12-19        │
├─

In [39]:
# Étape 4 : Sélection d'une option
options = stock.option_chain(selected_date)
while True:
    try:
        option_type = input("Option type (Call/Put) (default: Call): ").strip().lower() or "call"
        if option_type == "call":
            option_data = options.calls
        elif option_type == "put":
            option_data = options.puts
        else:
            print("Error: Please enter 'Call' or 'Put'.")
            continue

        option_data.reset_index(drop=True, inplace=True)
        option_data.insert(0, "No.", range(1, len(option_data) + 1))
        data_list = option_data.values.tolist()
        headers = option_data.columns.tolist()
        print("\nAvailable options for the selected type and date:")
        print(tb.tabulate(data_list, headers=headers, tablefmt="fancy_grid"))
        selected_option = option_data.iloc[int(input("Select an option (by No.) (default: 1): ") or 1) - 1]
        print(f"Selected option: {selected_option}")
        break
    except (IndexError, ValueError):
        print("Error: Please enter a valid number.")


Available options for the selected type and date:
╒═══════╤═════════════════════╤═══════════════════════════╤══════════╤═════════════╤════════╤════════╤══════════╤═════════════════╤══════════╤════════════════╤═════════════════════╤══════════════╤════════════════╤════════════╕
│   No. │ contractSymbol      │ lastTradeDate             │   strike │   lastPrice │    bid │    ask │   change │   percentChange │   volume │   openInterest │   impliedVolatility │ inTheMoney   │ contractSize   │ currency   │
╞═══════╪═════════════════════╪═══════════════════════════╪══════════╪═════════════╪════════╪════════╪══════════╪═════════════════╪══════════╪════════════════╪═════════════════════╪══════════════╪════════════════╪════════════╡
│     1 │ AAPL250620C00005000 │ 2024-12-04 18:35:54+00:00 │        5 │      237.41 │ 239.75 │ 243.55 │  0       │         0       │        5 │            879 │            3.79297  │ True         │ REGULAR        │ USD        │
├───────┼─────────────────────┼──────────

In [40]:
# Étape 5 : Calcul de la volatilité historique et des rendements
start_sub_period = default_start_date.strftime('%Y-%m-%d')
end_sub_period = today.strftime('%Y-%m-%d')
returns, mean, std = calculate_stat_returns(prices, start_sub_period, end_sub_period)

In [41]:
# Étape 6 : Calcul du temps à maturité
T = max((pd.to_datetime(selected_date) - today).days / 252, 0.001)

In [42]:
# Étape 7 : Simulation Monte Carlo
print("\nRunning Monte Carlo simulation...")
monte_carlo_price = monte_carlo_simulation(
    S=prices['Close'].iloc[-1],
    K=selected_option['strike'],
    T=T,
    r=0.03,  # Risk-free rate
    sigma=std,
    nb_simulation=1000,
    option_type=option_type
)


Running Monte Carlo simulation...


In [43]:
# Étape 8 : Calcul avec le modèle Black-Scholes
print("Calculating price using Black-Scholes model...")
black_scholes_price = black_scholes(
    S=prices['Close'].iloc[-1],
    K=selected_option['strike'],
    T=T,
    r=0.03,
    sigma=std,
    option_type=option_type
)

Calculating price using Black-Scholes model...


In [44]:
# Étape 9 : Affichage des résultats
print("\n### Results ###")
print(f"Market price of the selected option: {selected_option['lastPrice']:.2f} USD")
print(f"Monte Carlo price: {monte_carlo_price:.2f} USD")
print(f"Black-Scholes price: {black_scholes_price:.2f} USD")


### Results ###
Market price of the selected option: 49.49 USD
Monte Carlo price: 41.11 USD
Black-Scholes price: 42.47 USD


In [45]:
# Étape 10 : Comparaisons avec le prix du marché
print("\nPrice Comparison:")
if monte_carlo_price > selected_option['lastPrice']:
    print("The option appears to be undervalued according to the Monte Carlo method.")
else:
    print("The option appears to be overvalued according to the Monte Carlo method.")

if black_scholes_price > selected_option['lastPrice']:
    print("The option appears to be undervalued according to the Black-Scholes model.")
else:
    print("The option appears to be overvalued according to the Black-Scholes model.")


Price Comparison:
The option appears to be overvalued according to the Monte Carlo method.
The option appears to be overvalued according to the Black-Scholes model.
